In [1]:
import numpy as np
from pyvi.ViTokenizer import ViTokenizer
import nltk
import os
import re 
from underthesea import word_tokenize
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\drawt\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [121]:
import numpy as np
from pyvi.ViTokenizer import ViTokenizer
import re
class Text_Extraction:
    def __init__(self):
        pass
    def ExtractNameEntity(self, text):
        pattern = re.compile(r'<ENAMEX TYPE="(.+?)">(.+?)<')
        match = re.search(pattern, text)
        if match:
            return match.group(1), match.group(2)
        else:
            return None, None
    def Remove_punctuation(self, text):
        text = re.sub(r'[^\w\s]', '', text)
        return text
    def TextConstructer(self, text):
        First = text.split('<ENAMEX TYPE="')[0]
        ft = ViTokenizer.tokenize(First)
        ft = self.Remove_punctuation(ft)
        name_lib_first = []
        for word in ft.split():
            name_lib_first.append((word, 'I'))
        NE, word = self.ExtractNameEntity(text)
        if NE is None:
            return name_lib_first
        word = self.Remove_punctuation(word)
        lword = ViTokenizer.tokenize(word).split()
        name_lib = []
        for i, word in enumerate(lword):
            name_lib.append((word, NE))
        return name_lib_first + name_lib
    def remove_inner_tag(self, text):
        #this function take text, remove inner tag which have outer tag and keep outer tag and all tag that dont have inner tag
        pattern = re.compile(r'<ENAMEX TYPE="[^>]*">[^>]*<ENAMEX TYPE="(.+?)">(.+?)<\/ENAMEX>')
        match = re.search(pattern, text)
        if match:
            innter_text = match.group(2)
            inner_type = match.group(1)
            return text.replace(f'<ENAMEX TYPE="{inner_type}">{innter_text}</ENAMEX>', innter_text)
        return text
    def text_extract(self,text):
        text = self.remove_inner_tag(text)
        list_text = text.split('/ENAMEX>')
        list_t = []
        for text in list_text:
            list_t += self.TextConstructer(text)
        return list_t
    def list_text_extract(self,list_text):
        list_t = []
        for text in list_text:
            list_t.append(self.text_extract(text))
        return list_t

In [122]:
base_path = './data/'
list_file = ['VLSP2018-NER-dev', 'VLSP2018-NER-test', 'VLSP2018-NER-train']

In [123]:
def read_file(file_name):
    list_text = []
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = filter(None, (line.rstrip() for line in f))
        for line in lines:
            list_text.append(line)
    return list_text

In [124]:
def load_file(path):
    list_file = os.listdir(path)
    list_text = []
    for file_name in list_file:
        texts = read_file(os.path.join(path, file_name))
        list_text += texts
    return list_text

In [125]:
def load_data(path):
    list_file = os.listdir(path)
    list_text = []
    for file_name in list_file:
        list_text += load_file(os.path.join(path, file_name))
    return list_text

In [126]:

train = load_data(os.path.join(base_path, list_file[2]))


In [187]:
import pycrfsuite
from sklearn.metrics import classification_report
class NER_model:
    def __init__(self):
        pass
    def pos_tag(self, docs):
        data = []
        for i, doc in enumerate(docs):
            tokens = [t for t, label in doc]
            tagged = nltk.pos_tag(tokens)
            data.append([(w, pos, label) for (w, label), (word, pos) in zip(doc, tagged)])
        return data
    def word2feature(self, doc, i):
        word = doc[i][0]
        postag = doc[i][1]
        #common features for all words
        features = {
            'bias': 1.0,
            'word.lower()': word.lower(),
            'word[-3:]': word[-3:],
            'word[-2:]': word[-2:],
            'word.isupper()': word.isupper(),
            'word.istitle()': word.istitle(),
            'word.isdigit()': word.isdigit(),
            'postag': postag,
            'postag[:2]': postag[:2],
        }
        if i>0:
            word1 = doc[i-1][0]
            postag1 = doc[i-1][1]
            features.update({
                '-1:word.lower()': word1.lower(),
                '-1:word.istitle()': word1.istitle(),
                '-1:word.isupper()': word1.isupper(),
                '-1:postag': postag1,
                '-1:postag[:2]': postag1[:2],
            })
        else:
            features['BOS'] = True
        if i<len(doc)-1:
            word1 = doc[i+1][0]
            postag1 = doc[i+1][1]
            features.update({
                '+1:word.lower()': word1.lower(),
                '+1:word.istitle()': word1.istitle(),
                '+1:word.isupper()': word1.isupper(),
                '+1:postag': postag1,
                '+1:postag[:2]': postag1[:2],
            })
        else:
            features['EOS'] = True
        return features
    def extract_features(self, docs):
        return [self.word2feature(docs, i) for i in range(len(docs))]
    def get_labels(self, doc):
        return [label for (token, postag, label) in doc]
    def fit(self, X_train, y_train, model_name = 'crf.model', c1 = 0.1, c2 = 0.01, max_iterations = 50, possible_transitions = True, verbose = True):
        trainer = pycrfsuite.Trainer(verbose=verbose)
        for xseq, yseq in zip(X_train, y_train):
            trainer.append(xseq, yseq)
        trainer.set_params({
            'c1': c1,
            'c2': c2,
            'max_iterations': max_iterations,
            'feature.possible_transitions': possible_transitions
        })
        trainer.train(model_name)
        print('Training done! Model was saved in {}'.format(model_name))
    def get_XY(self, docs):
        docs = self.pos_tag(docs)
        return [self.extract_features(doc) for doc in docs], [self.get_labels(doc) for doc in docs]
    def score(self, X_test, y_test, model_name = 'crf.model'):
        tagger = pycrfsuite.Tagger()
        tagger.open(model_name)
        y_pred = [tagger.tag(xseq) for xseq in X_test]
        y_pred = np.array(y_pred)
        y_test = np.array(y_test)
        return np.mean(y_pred == y_test)
    def predict(self, X_test, model_name = 'crf.model'):
        tagger = pycrfsuite.Tagger()
        tagger.open(model_name)
        y_pred = [tagger.tag(xseq) for xseq in X_test]
        return y_pred
    def classification_report(self, x_test, y_test, model_name = 'crf.model'):
        tagger = pycrfsuite.Tagger()
        tagger.open(model_name)
    
        y_pred = [tagger.tag(xseq) for xseq in x_test]
        predictions = np.array([tag for row in y_pred for tag in row])
        truths = np.array([tag for row in y_test for tag in row])
        return classification_report(truths, predictions)
    def print_result(self, x_test,  model_name = 'crf.model'):
        tagger = pycrfsuite.Tagger()
        tagger.open(model_name)
        y_pred = [tagger.tag(xseq) for xseq in x_test]
        for i in range(len(x_test)):
            print('Sentence {}:'.format(i+1))
            for j in range(len(x_test[i])):
                print(x_test[i][j]['word.lower()'], y_pred[i][j])
            print('\n')


In [188]:
extractor = Text_Extraction()

In [189]:
#extractor.text_extract(train[20])
extractor.remove_inner_tag(train[20])

'Liên quan thuốc lá lậu, theo báo cáo <ENAMEX TYPE="ORGANIZATION">Oxford Economics</ENAMEX> 2014, Việt Nam đã trở thành thị trường tiêu thụ thuốc lá lậu hàng đầu trong số 14 quốc gia và vùng lãnh thổ <ENAMEX TYPE="LOCATION">châu Á</ENAMEX> được khảo sát (gồm: <ENAMEX TYPE="LOCATION">Indonesia</ENAMEX>, <ENAMEX TYPE="LOCATION">Thái Lan</ENAMEX>, <ENAMEX TYPE="LOCATION">Campuchia</ENAMEX>, <ENAMEX TYPE="LOCATION">Đài Loan</ENAMEX>, <ENAMEX TYPE="LOCATION">Lào</ENAMEX>, <ENAMEX TYPE="LOCATION">Úc</ENAMEX>, <ENAMEX TYPE="LOCATION">Philippines</ENAMEX>, <ENAMEX TYPE="LOCATION">Singapore</ENAMEX>, Việt Nam, <ENAMEX TYPE="LOCATION">Myanmar</ENAMEX>, <ENAMEX TYPE="LOCATION">Pakistan</ENAMEX>, <ENAMEX TYPE="LOCATION">Hồng Công</ENAMEX>, <ENAMEX TYPE="LOCATION">Malaysia</ENAMEX>, <ENAMEX TYPE="LOCATION">Brunei</ENAMEX>). Năm 2016 và 6 tháng đầu năm 2017, tình trạng buôn lậu thuốc lá có chiều hướng gia tăng. Theo <ENAMEX TYPE="ORGANIZATION">Hiệp hội Thuốc lá Việt Nam</ENAMEX>, năm 2016 và những t

In [190]:
docs = extractor.list_text_extract(train)

In [191]:
model = NER_model()
X_train, y_train = model.get_XY(docs)
model.fit(X_train, y_train, max_iterations=200)

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 78147
Seconds required: 0.539

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 151339.813123
Feature norm: 1.000000
Error norm: 122092.259442
Active features: 77664
Line search trials: 1
Line search step: 0.000002
Seconds required for this iteration: 0.348

***** Iteration #2 *****
Loss: 130598.173588
Feature norm: 1.244825
Error norm: 48095.338529
Active features: 74640
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.175

***** Iteration #3 *****
Loss: 123308.653071
Feature norm: 1.384545
Error norm: 35867.345925
Active features: 75457
Line search trials: 1
Line search step: 1.000000
Seconds required

In [192]:
valid = load_data(os.path.join(base_path, list_file[0]))
docs_valid = extractor.list_text_extract(valid)
X_valid, y_valid = model.get_XY(docs_valid)
y_val_pred = model.predict(X_valid)

In [193]:
print(model.classification_report(X_valid, y_valid))

               precision    recall  f1-score   support

            I       0.99      0.99      0.99    100573
     LOCATION       0.83      0.79      0.81      2946
MISCELLANEOUS       0.67      0.53      0.59       428
 ORGANIZATION       0.86      0.77      0.81      3903
       PERSON       0.88      0.81      0.84      2370

     accuracy                           0.97    110220
    macro avg       0.84      0.78      0.81    110220
 weighted avg       0.97      0.97      0.97    110220



In [194]:
test = load_data(os.path.join(base_path, list_file[1]))
docs_test = extractor.list_text_extract(test)
X_test, y_test = model.get_XY(docs_test)
model.score(X_test, y_test)

C:\Users\drawt\AppData\Local\Temp\ipykernel_6736\1967614520.py:76: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y_pred = np.array(y_pred)
C:\Users\drawt\AppData\Local\Temp\ipykernel_6736\1967614520.py:77: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y_test = np.array(y_test)


0.5382808830436825

In [195]:
print(model.classification_report(X_test, y_test))

               precision    recall  f1-score   support

            I       0.98      0.99      0.99    171889
     LOCATION       0.65      0.70      0.67      2471
MISCELLANEOUS       0.47      0.21      0.29       621
 ORGANIZATION       0.71      0.58      0.64      3975
       PERSON       0.75      0.59      0.66      3978

     accuracy                           0.97    182934
    macro avg       0.71      0.62      0.65    182934
 weighted avg       0.96      0.97      0.97    182934



In [205]:
print(model.classification_report(X_train, y_train))

               precision    recall  f1-score   support

            I       1.00      1.00      1.00    300605
     LOCATION       0.98      0.99      0.99      9131
MISCELLANEOUS       0.99      1.00      0.99      1856
 ORGANIZATION       0.99      0.99      0.99     10005
       PERSON       1.00      1.00      1.00      6905

     accuracy                           1.00    328502
    macro avg       0.99      1.00      0.99    328502
 weighted avg       1.00      1.00      1.00    328502



In [207]:
model.print_result(X_test[50:51])

Sentence 1:
tên_lửa I
9m317m I
được I
bắt_đầu I
được I
phát_triển I
từ I
những I
năm I
2000 I
và I
lần I
đầu_tiên I
được I
giới_thiệu I
tại I
cuộc I
triển_lãm I
quốc_tế I
defendory MISCELLANEOUS
international MISCELLANEOUS
2006 I
ở I
hy_lạp LOCATION
có I
tầm I
phóng I
được I
nâng I
lên I
tới I
70km I
độ I
cao I
35km I
với I
vận_tốc I
siêu_thanh I
gần I
mach I
10 I
tốc_độ I
đánh I
chặn I
nhanh I
hơn I
cả I
s I
400 I
500 I


